## Prepare

In [38]:
import os
import json
import ipywidgets as widgets
from ipywidgets import Text, HTML, HBox, Layout, VBox
from ipyleaflet import (
    Map,
    Marker,
    TileLayer, ImageOverlay,
    Polyline, Polygon, Rectangle, Circle, CircleMarker,
    GeoJSON,
    DrawControl,
    WidgetControl,
    basemaps,
    Popup,
    Icon,
    MarkerCluster
)

from math import sqrt

In [39]:
from fabrictestbed.slice_manager import SliceManager, Status

credmgr_host = os.environ['FABRIC_CREDMGR_HOST']
orchestrator_host = os.environ['FABRIC_ORCHESTRATOR_HOST']
print(f"CM Host: {credmgr_host} Orchestrator Host: {orchestrator_host}")

slice_manager = SliceManager(oc_host = orchestrator_host, cm_host = credmgr_host, project_name='all', scope='all')
slice_manager.initialize()

print("Token location: {}".format(slice_manager.token_location))
print()

#Cofigure SSH Key
ssh_key = None
with open ("/home/fabric/.ssh/id_rsa.pub", "r") as myfile:
    ssh_key=myfile.read().strip()

    
try:
    id_token, refresh_token = slice_manager.refresh_tokens()
except Exception as e:
    print("Exception occurred while getting tokens:{}".format(e))

fabric_refresh_token=slice_manager.get_refresh_token()
print()
print("New Refresh Token: {}".format(fabric_refresh_token))
print()
print("Stored new Refresh Token")
%store fabric_refresh_token
print()

CM Host: beta-2.fabric-testbed.net Orchestrator Host: beta-7.fabric-testbed.net
Token location: /home/fabric/.tokens.json


New Refresh Token: NB2HI4DTHIXS6Y3JNRXWO33OFZXXEZZPN5QXK5DIGIXTKMDFGI3DAYRRMQ2DOMBTHA2GMZJXHEYWGZRSMYZGEZDDGY2DGOB7OR4XAZJ5OJSWM4TFONUFI33LMVXCM5DTHUYTMMRWG4YTKNJRGIYDINJGOZSXE43JN5XD25RSFYYCM3DJMZSXI2LNMU6TQNRUGAYDAMBQ

Stored new Refresh Token
Stored 'fabric_refresh_token' (str)



In [40]:
site_dict = {'UKY': (38.03154264461649, -84.50343326019218), 'RENC': (35.93996154497003, -79.01809264307425),
             'LBNL': (38.94977311613871, -122.61289106281062), 'STAR': (41.860757376744274, -87.63103410014864)}

## Create experiment with map

In [41]:
from fabrictestbed.slice_editor import ExperimentTopology, Capacities, ComponentType, LinkType, Layer

# Create Experiment Topology
experiment = ExperimentTopology()

slice_name = "MapDemoSlice"

## Functions

In [42]:
node_name = widgets.Text(
    placeholder='enter name',
    description='Node Name:',
    disabled=False
)

site_option = widgets.Text(
    value='UKY',
    description='Site:',
    disabled=False
)

core_slider = widgets.IntSlider(min=2, max=32, step=2, value=4, description='Cores')
ram_slider = widgets.IntSlider(min=2, max=64, step=2, value=16, description='RAM')
disk_slider = widgets.FloatLogSlider(base=2, min=5, max=17, step=1, value=128,description='Disk')

img_ref_select = widgets.Select(
    options=['qcow2', 'qchicken2', 'qpig2'],
    value='qcow2',
    description='Image Type:',
    disabled=False
)

img_type_select = widgets.Select(
    options=['default_ubuntu_20', 'default_arch_20', 'default_kali_21'],
    value='default_ubuntu_20',
    description='Image Reference:',
    disabled=False
)

## Node creation map

In [43]:
from ipyleaflet import DrawControl, CircleMarker

#initialize map 
canvas = Map(basemap = basemaps.Esri.WorldStreetMap, center=(38.12480976137421, -95.7129), zoom = 3.8)

#draw control
dc = DrawControl(circlemarker={}, polygon={})
canvas.add_control(dc)

#initialize widget control
init = [HTML('Click on a site to create a new node')]
my_layout = Layout(height='50%', overflow_y= 'auto', padding = "4px" )
html_box = VBox(init , layout = my_layout )
control = WidgetControl(widget = html_box, position='topright')
canvas.add_control(control)


icon = Icon(icon_url='https://jupyter-beta.fabric-testbed.net/user/nannkat@bu.edu/files/jupyter-examples/nanna/map_experiments/circle.png?_xsrf=2%7C4b270639%7C01ca1ecbc98a0090a488f8688b3a0dd1%7C1625580900', icon_size=[15,15])

#function to find closest site
def find_closest(coordinates, sitedict):
    closest_site = 'UKY'
    closest_value = 500000
    for site,coors in sitedict.items():
        distance = sqrt(pow((coordinates[1] - coors[0]), 2) + pow((coordinates[0] - coors[1]), 2))
        if distance < closest_value:
            closest_site = site
            closest_value = distance
    return closest_site

markers = {}
circles = {}

#add node btn
def add_node(self):
    n1 = experiment.add_node(name=node_name.value, site=site_option.value)
    cap = Capacities().set_fields(core=core_slider.value, ram=ram_slider.value, disk=disk_slider.value)
    n1.set_properties(capacities=cap)
    node_name.value = ''
    html_box.children = init
    core_slider.value = 4
    ram_slider.value = 16
    disk_slider.value = 128
    

add_node_request = widgets.Button(
    description='Add Node',
    disabled=False,
    tooltip='click to build node',
)

add_node_request.on_click(add_node)

#cancel btn 
def esc(self):
    html_box.children = init
    node_name.value = ''
    canvas.remove_layer(markers[site_option.value])
    
exit_btn = widgets.Button(
    description='Cancel',
    disabled=False,
    tooltip='click to exit',)

exit_btn.on_click(esc)

#node creation menu
def node_dashboard(site_name):
    header = HTML('<center><b>New Node</b></center>')
    site_option.value = site_name
    btns = HBox([VBox([add_node_request]), VBox([exit_btn])])
    content = [header,node_name, site_option, core_slider, ram_slider, disk_slider, btns]
    html_box.children = content


#node creation on click    
def new_node(**kwargs):
    coors = kwargs['coordinates']
    mark = Marker(location = coors, draggable = False)
    mark.on_click(remove_marker)
    canvas.add_layer(mark)
    site_name = find_closest((coors[1],coors[0]), site_dict)
    markers[site_name] = mark
    node_dashboard(site_name)

#remove the used marker
def remove_marker(**kwargs):
    coors = kwargs['coordinates']
    site_name = find_closest((coors[1],coors[0]), site_dict)
    marker = markers[site_name]
    canvas.remove_layer(marker)

#populate map
for name, coordinates in site_dict.items():
    m = Marker(location = coordinates, icon = icon, draggable = False, title = name)
    circles[name] = m
    m.on_click(new_node)
    canvas.add_layer(m)

canvas

Map(center=[38.12480976137421, -95.7129], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_…

In [37]:
for node in experiment.nodes:
    print(node)